# This notebook serves as an example of how to create AutoTST objects and how to create 3D geometries

In [1]:
#General imports
import os, sys
import logging
from copy import deepcopy
import numpy as np
import pandas as pd
from multiprocessing import Process

#AutoTST imports
from autotst.species import Species, Conformer
from autotst.reaction import Reaction, TS 
from autotst.calculator.gaussian import Gaussian
from autotst.calculator.statmech import StatMech
from autotst.job.job import Job


RDKit WARNING: [15:23:34] Enabling RDKit 2019.09.2 jupyter extensions


# Creating `Species` and `Conformer` objects

Each stable reactant and product species are generated as `Species` objects. These objects can contain many `Conformer` objects that can represent multiple resonance structures. 
`Species` and `Conformer` objects can be initialized as follows:
- `Species(["SMILES_STRING"])`
- `Conformer("SMILES_STRING")`

For a `Species` object, it can take lists of SMILES strings because there may be multiple resonance structures for a given SMILES structure. However, `Conformer` objects represent only a single isomer.
`Species` objects can contain multiple `Conformer` and can be 

In [2]:
species = Species(["[CH2]C=C(C)C"])
print("For {} there are {} resonance structures".format(species, len(species.conformers)))
print("These structures are:")
for smiles in species.conformers.keys():
    print ("\t- {}".format(smiles))

For <Species "[CH2]C=C(C)C / C=C[C](C)C"> there are 2 resonance structures
These structures are:
	- [CH2]C=C(C)C
	- C=C[C](C)C


In [3]:
species.conformers

{'[CH2]C=C(C)C': [<Conformer "[CH2]C=C(C)C">],
 'C=C[C](C)C': [<Conformer "C=C[C](C)C">]}

When looking at a specific `Conformer` object, you need to know that a `Species` contants a dictionary in `Species.conformers`. The keys are the possible SMILES strings associated with the species and the values are lists of `Conformer` objects for that specific SMILES. When initialized, these lists will be length 1 but will be extended if you generate more conformers. Below is how to view a single `Conformer` object 

In [4]:
conformer = species.conformers[smiles][0]
conformer.view()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Creating `Reaction` and `TS` objects
For this example we will be looking at a hydrogen abstraction reaction by peroxyl radical of 2-methylbut-2-ene

First, you need to initialize the autotst `Reaction` object as done below. This can be done by using either reaction strings that look like `r1+r2_p1+p2` where `r1`, `r2`, `p1`, and `p2` are smiles strings for the molecules involved in the reaction. In addition, not all reaction need to by bimolecular. AutoTST currently supports reactions of the following reaction families:
- Hydrogen Abstraction (`H_Abstraction`: `r1+r2_p1+p2`)
- Intra hydrogen migration (`intra_H_migration`: `r1_p1`)
- R addition to multiple bond (`R_Addition_MultipleBond`: `r1+r2_p1`)

You can specify a `reaction_family` in a `Reaction` object, however, it is not needed as AutoTST will attempt to match the reaction provided to one of the three supported reaction families.
Alternatively, you can intialize a `Reaction` object using an `RMGReaction` object. This can be done as follows:

`rxn = Reaction(rmg_reaction=RMGReaction())`

The initialization of the reaction will also create a forward and a reverse transition state geometry. And these can be visualized using `py3dmol`.

`Reaction` objects are similar to `Species` objects in that they both contain a dictionary corresponding to their conformers. The `Reaction.ts` is the dictionary of transition states for a reaction just like the `Species.conformers` is the dictionary of conformers for a species. The only difference is the keys for the `Reaction.ts` are simply `"forward"` and `"reverse"` to denote the `TS` generated in either direction

In [5]:
rxn = Reaction(label="CC=C(C)C+[O]O_[CH2]C=C(C)C+OO")

In [6]:
transitionstates = rxn.ts["reverse"] #the rxn.ts is a dictionary with keys being "forward" and "reverse"
ts = transitionstates[0] #transitionstates is a list of TS objects, this list is currently length 1
rxn.ts

reaction.py:166 load_databases INFO Loading RMG database from '/Users/nathan/Code/RMG-database/input'
thermo.py:902 load_libraries INFO Loading thermodynamics library from primaryThermoLibrary.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:902 load_libraries INFO Loading thermodynamics library from thermo_DFT_CCSDTF12_BAC.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:902 load_libraries INFO Loading thermodynamics library from CBS_QB3_1dHR.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:921 load_groups INFO Loading thermodynamics group database from /Users/nathan/Code/RMG-database/input/thermo/groups...
transport.py:294 load_groups INFO Loading transport group database from /Users/nathan/Code/RMG-database/input/transport/groups...
statmech.py:553 load_groups INFO Loading frequencies group database from /Users/nathan/Code/RMG-database/input/statmech/groups...
base.py:215 load INFO Loading transitions state f

{'forward': [<TS "CC=C(C)C.[O]O">], 'reverse': [<TS "OO.[CH2]C=C(C)C">]}

# Editing geometries of `Conformer` and `TS` objects

AutoTST allows you to edit the following features of a `Conformer` or a `TS` object:
- Bond length
- Angles
- Dihedrals
- CisTrans bond orientation
- Sterocenter orientation

The definitions for all of these objects can be found in `geometry.py`.

In [7]:
conformer = Conformer("ClC=C(O)C(N)Cl")
conformer.view()
print("{} has the following geometries:".format(conformer))
print("")
print("Bonds")
for bond in conformer.bonds:
    print("\t- {}: {}".format(bond, bond.index))
    
print("")
print("Angles")
for angle in conformer.angles:
    print("\t- {}: {}".format(angle, angle.index))
    
print("")
print("Dihedrals")
for torsion in conformer.torsions:
    print("\t- {}: {}".format(torsion, torsion.index))
    
print("")
print("CisTrans Bonds")
for cistran in conformer.cistrans:
    print("\t- {}: ".format(cistran, cistran.index))
    
print("")
print("Chiral Centers")
for chiral_center in conformer.chiral_centers:
    print("\t- {}: {}".format(chiral_center, chiral_center.index))


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

<Conformer "ClC=C(O)C(N)Cl"> has the following geometries:

Bonds
	- <Bond "(0, 4)">: 0
	- <Bond "(1, 6)">: 1
	- <Bond "(2, 5)">: 2
	- <Bond "(2, 11)">: 3
	- <Bond "(3, 4)">: 4
	- <Bond "(3, 8)">: 5
	- <Bond "(3, 9)">: 6
	- <Bond "(4, 5)">: 7
	- <Bond "(4, 7)">: 8
	- <Bond "(5, 6)">: 9
	- <Bond "(6, 10)">: 10

Angles
	- <Angle "(0, 4, 3)">: 0
	- <Angle "(0, 4, 5)">: 1
	- <Angle "(0, 4, 7)">: 2
	- <Angle "(1, 6, 5)">: 3
	- <Angle "(1, 6, 10)">: 4
	- <Angle "(2, 5, 4)">: 5
	- <Angle "(2, 5, 6)">: 6
	- <Angle "(3, 4, 5)">: 7
	- <Angle "(3, 4, 7)">: 8
	- <Angle "(4, 3, 8)">: 9
	- <Angle "(4, 3, 9)">: 10
	- <Angle "(4, 5, 6)">: 11
	- <Angle "(5, 2, 11)">: 12
	- <Angle "(5, 4, 7)">: 13
	- <Angle "(5, 6, 10)">: 14
	- <Angle "(8, 3, 9)">: 15

Dihedrals
	- <Torsion "(11, 2, 5, 6)">: 0
	- <Torsion "(9, 3, 4, 7)">: 1
	- <Torsion "(7, 4, 5, 6)">: 2

CisTrans Bonds
	- <CisTrans "[4, 5, 6, 10] - Z">: 

Chiral Centers
	- <ChiralCenter "4 - ?">: 0


In addition, you can set variables for each of these objects. This next section of the notebook will walk though how to do all of this.

In [8]:
conformer.set_bond_length(bond_index=3, length=3) #Length is specified in angstroms
conformer.view()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [9]:
conformer.set_angle(angle_index=7, angle=45.) #Angles are specified in degrees
conformer.view()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [10]:
conformer.set_torsion(torsion_index=0, dihedral=180)
conformer.view()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [11]:
conformer.set_cistrans(cistrans_index=0, stero="z") #options for stero are E and Z
conformer.view()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [12]:
conformer.set_chirality(chiral_center_index=0, stero="r")
conformer.view()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## AND all of these features can be extended to `TS` objects as well 

# Now, let's look at how to generate ensembles of conformers for `Species` and `Reactions` using AutoTST

Both `Species` and `Reaction` objects have a built-in method called `generate_conformers` and this method takes one argument of an ASE calculator object. This method will then generate all possible combinations of dihedrals, CisTrans bonds, and chiral centers to identify every possible conformer. All of these conformers are then optimized using ASE's BFGS optimizer and the calculator that a user provides. From this, a list of conformers within 1 kcal/mol are generated and added to the `Species.conformers` or the `Reaction.ts` dict. Below are a few examples.

In [13]:
from ase.calculators.emt import EMT
species = Species(["CCCC"])
species.generate_conformers(EMT())
species.conformers

systematic.py:272 systematic_search INFO There are 3 possible conformers to investigate...
systematic.py:388 systematic_search INFO We have identified 1 unique, low-energy conformers for <Conformer "CCCC">


{'CCCC': [<Conformer "CCCC">]}

In [14]:
reaction = Reaction("C+[O]O_[CH3]+OO")
reaction.generate_conformers(EMT())
reaction.ts

reaction.py:321 get_labeled_reaction INFO Trying to match reacction to <ReactionFamily "H_Abstraction">
reaction.py:349 get_labeled_reaction INFO Matched reaction to H_Abstraction family
reaction.py:321 get_labeled_reaction INFO Trying to match reacction to <ReactionFamily "R_Addition_MultipleBond">
reaction.py:321 get_labeled_reaction INFO Trying to match reacction to <ReactionFamily "intra_H_migration">
reaction.py:332 get_labeled_reaction ERROR Couldn't match <Molecule "C"> + <Molecule "[O]O"> <=> <Molecule "[CH3]"> + <Molecule "OO"> to intra_H_migration, trying different combination...
reaction.py:321 get_labeled_reaction INFO Trying to match reacction to <ReactionFamily "H_Abstraction">
reaction.py:349 get_labeled_reaction INFO Matched reaction to H_Abstraction family
reaction.py:321 get_labeled_reaction INFO Trying to match reacction to <ReactionFamily "R_Addition_MultipleBond">
reaction.py:321 get_labeled_reaction INFO Trying to match reacction to <ReactionFamily "intra_H_migrat

{'forward': [<TS "C.[O]O">, <TS "C.[O]O">],
 'reverse': [<TS "OO.[CH3]">, <TS "OO.[CH3]">]}

For right now, we're going to be looking at the first conformers for the `Species` and `Reaction` listed above.

In [15]:
ts = reaction.ts["forward"][0]
conformer = species.conformers["CCCC"][0]

# Now, to look at writing input files for `Conformer` and `TS` objects

This is realitively easy, you just need to use the `Gaussian` calculator object to do all of this. This starts with an initialzation of the object followed by calling it's methods on different objects to get ASE calculator objects that can write your geometries.

The main methods of the `Gaussian` calculator are:
- For species conformers 
    - `get_conformer_calc`
- For transition states
    - `get_shell_calc`
    - `get_center_calc`
    - `get_overall_calc`
    - `get_irc_calc`
- For both
    - `get_rotor_calc`

In [16]:
gaussian = Gaussian()

For `Conformer` objects

In [17]:
gaussian.conformer = conformer
try:
    calc = gaussian.get_conformer_calc()
    calc.write_input(conformer.ase_molecule)
    f = open(calc.label + ".com", "r")
    print(calc.label)
    print("")
    for line in f.readlines():
        print(line[:-1])
except:
    print("Sorry, it seems that you're running on a system without Gaussian...")
    print("The rest of the gaussian tutorial may not work...")

Sorry, it seems that you're running on a system without Gaussian...
The rest of the gaussian tutorial may not work...


For `TS` objects

In [18]:
gaussian.conformer = ts
try:
    calc = gaussian.get_shell_calc()
    calc.write_input(ts.ase_molecule)
    f = open(calc.label + ".com", "r")
    print(calc.label)
    print("")
    for line in f.readlines():
        print(line[:-1])
except:
    print("Sorry, it seems that you're running on a system without Gaussian...")
    print("The rest of the gaussian tutorial may not work...")

Sorry, it seems that you're running on a system without Gaussian...
The rest of the gaussian tutorial may not work...


In [19]:
try:
    calc = gaussian.get_overall_calc()
    calc.write_input(ts.ase_molecule)
    f = open(calc.label + ".com", "r")
    print(calc.label)
    print("")
    for line in f.readlines():
        print(line[:-1])
except:
    print("Sorry, it seems that you're running on a system without Gaussian...")
    print("The rest of the gaussian tutorial may not work...")

Sorry, it seems that you're running on a system without Gaussian...
The rest of the gaussian tutorial may not work...


In [20]:
try:
    calc = gaussian.get_irc_calc()
    calc.write_input(ts.ase_molecule)
    f = open(calc.label + ".com", "r")
    print(calc.label)
    print("")
    for line in f.readlines():
        print(line[:-1])
except:
    print("Sorry, it seems that you're running on a system without Gaussian...")
    print("The rest of the gaussian tutorial may not work...")

Sorry, it seems that you're running on a system without Gaussian...
The rest of the gaussian tutorial may not work...


For hindered rotor calculations

In [21]:
torsion = conformer.torsions[0]
gaussian.conformer = conformer
try:
    calc = gaussian.get_rotor_calc(torsion_index=torsion.index)
    calc.write_input(conformer.ase_molecule)
    f = open(calc.label + ".com", "r")
    print(calc.label)
    print("")
    for line in f.readlines():
        print(line[:-1])
except:
    print("Sorry, it seems that you're running on a system without Gaussian...")
    print("The rest of the gaussian tutorial may not work...")

Sorry, it seems that you're running on a system without Gaussian...
The rest of the gaussian tutorial may not work...


In [22]:
torsion = ts.torsions[0]
gaussian.conformer = ts
try:
    calc = gaussian.get_rotor_calc(torsion_index=torsion.index)
    calc.write_input(ts.ase_molecule)
    f = open(calc.label + ".com", "r")
    print(calc.label)
    print("")
    for line in f.readlines():
        print(line[:-1])
except:
    print("Sorry, it seems that you're running on a system without Gaussian...")
    print("The rest of the gaussian tutorial may not work...")

Sorry, it seems that you're running on a system without Gaussian...
The rest of the gaussian tutorial may not work...


At this point, you now have a way to have input files for quantum chemistry optimizations to be automatically written for you. And you can have all of these run automatically using the AutoTST `Job` class